# Install Packages

In [1]:
# ! pip -q install openai agno
# ! pip install pandas
# ! pip install python-dotenv
# ! pip install duckduckgo-search
# ! pip install -U tavily-python
# ! pip install tantivy
# ! pip install openai newspaper4k lxml_html_clean
# ! pip install pydantic-ai


# Load Secrets

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path='../backend/.env')  # Adjust if not running from 'notebooks/'


# Access the variables
openai_key = os.getenv("OPENAI_API_KEY")
gmail_user = os.getenv("GMAIL_ADDRESS")
gmail_pass = os.getenv("GMAIL_APP_PASSWORD")
tavily_key = os.getenv("TAVILY_API_KEY")



print("Loaded API Key:", bool(openai_key))  # Should print True if loaded


Loaded API Key: True


In [3]:
from pydantic import BaseModel
from typing import List
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.common_tools.tavily import tavily_search_tool
import smtplib
from email.mime.text import MIMEText
from chromadb import Client

# Output Schema

In [4]:
from pydantic import BaseModel
from typing import List

# ---------------------------
# Structured Output Models
# ---------------------------
class AnswerSource(BaseModel):
    answer: str
    sources: List[str]

class FinalOutput(BaseModel):
    rag: AnswerSource
    web: AnswerSource
    final_summary: str


In [5]:
# ---------------------------
# Initialize Model & Agents
# ---------------------------
model = OpenAIModel("gpt-4")

# Vector DB
chroma_client = Client()
collection = chroma_client.get_or_create_collection(name="knowledge_base")


# Team Structure:

In [6]:
# Specialized Agents
rag_agent = Agent(
    model=model,
    system_prompt="You are a knowledge specialist. Use the RAG system to answer queries and provide sources."
)

web_agent = Agent(
    model=model,
    system_prompt="You are a research specialist. Conduct web searches to answer queries and provide sources."
)

email_agent = Agent(
    model=model,
    system_prompt="You are an email specialist. Draft and send emails based on provided content."
)

# Tools

In [ ]:
# Define tools first
async def query_rag(ctx: RunContext[None], query: str) -> AnswerSource:
    results = collection.query(query_texts=[query], n_results=3)
    documents = results["documents"][0]
    sources = results["metadatas"][0]
    answer = " ".join(documents)
    return AnswerSource(answer=answer, sources=sources)

async def send_email(ctx: RunContext[None], content: str) -> str:
    sender_email = gmail_user
    receiver_email = "nayeem60151126@gmail.com"
    password = gmail_pass

    message = MIMEText(content)
    message["Subject"] = "AI Summary"
    message["From"] = sender_email
    message["To"] = receiver_email

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.send_message(message)
        return "Email sent successfully."
    except Exception as e:
        return f"Failed to send email: {e}"


# Team Configuration:

In [9]:
team_lead_agent = Agent(
    model=model,
    system_prompt="Coordinate RAG and web search, summarize findings, and optionally send via email.",
    output_type=FinalOutput,
    tools=[
        query_rag,
        send_email,
        tavily_search_tool(api_key=tavily_key)
    ]
)


# Example Use

In [10]:
# ---------------------------
# Execution Function
# ---------------------------
import asyncio

async def main():
    user_query = "What are the latest developments in AI and LLMs?"

    # Run the Team Lead agent
    result = await team_lead_agent.run(user_query)
    output = result.output

    # Display structured outputs
    print("RAG Answer:", output.rag.answer)
    print("RAG Sources:", output.rag.sources)
    print("Web Answer:", output.web.answer)
    print("Web Sources:", output.web.sources)
    print("Final Summary:", output.final_summary)

    # Ask user for email confirmation
    confirmation = input("Would you like to send this summary via email? (yes/no): ").strip().lower()
    if confirmation == "yes":
        email_result = await send_email(None, output.final_summary)
        print(email_result)
    else:
        print("Email not sent.")

# Run the async main function (for notebook use)
await main()

RAG Answer: 
RAG Sources: []
Web Answer: Generative AI and large language models (LLMs) are at the center of recent advancements in AI. These LLMs, which were traditionally used for processing and understanding text, have now evolved to comprehend images and are becoming proficient in understanding speech and video data. They are now being used to handle complex logical tasks and step-by-step reasoning processes. Additionally, these models can understand diverse forms of information beyond just text. These advancements extend beyond professional settings, impacting personal lives by integrating into everyday tasks such as communication, education, and personal assistance. In the enterprise sector, LLMs are used for customer support, chatbots, internal knowledge retrieval, content generation, marketing, coding automation, and business intelligence. They can even help companies with context-aware recommendations, data insights, process optimizations, compliance, and strategic planning.
W